In [1]:
import os

In [2]:
%pwd

'c:\\Python_Projects\\DL\\deepCNNClassifier\\research'

In [3]:
os.chdir('../')

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path
    
    

In [11]:
from deepClassifier.utils import *
from deepClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        
        model_check_point = os.path.dirname(self.config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), Path(model_check_point)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        
        
        return prepare_callbacks_config
        
        
        
        
    

In [9]:
import os
import tensorflow as tf
import time



class PrepareCallback:
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
            )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ] 